In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision
from torch.autograd import Variable
import torch.optim as optim

use_gpu = True if torch.cuda.is_available() else False

# trained on high-quality celebrity faces "celebA" dataset
# this model outputs 512 x 512 pixel images
model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
                       'PGAN', model_name='celeba',
                       pretrained=True, useGPU=use_gpu)
# this model outputs 256 x 256 pixel images
# model = torch.hub.load('facebookresearch/pytorch_GAN_zoo:hub',
#                        'PGAN', model_name='celebAHQ-256',
#                        pretrained=True, useGPU=use_gpu)


# Construct latent vector z. Compute and visualize G(z)

In [ ]:
G = model.netG
z = torch.zeros(1,model.config.noiseVectorDim)
gen_image = G(z)


grid = torchvision.utils.make_grid(gen_image.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())
# plt.show()

# Compressed Sensing 

## Using Gradient Descent

In [ ]:
G = model.netG
mse = torch.nn.MSELoss()

G0 = G(torch.zeros(1,model.config.noiseVectorDim))
d_image = G0.numel()
x = G0.reshape(d_image)

f = 0.2 #compression rate
print('Compression rate is ', f)
m_image = int(f*d_image)
print('Number of measurements is ',m_image, ' for signal of length ', d_image)


# random Gaussian measurement matrix : A
A = torch.randn(m_image, d_image)
y = torch.matmul(A,x)

# optimization min ||AG(z)-AG0||
z = Variable(torch.randn(1,model.config.noiseVectorDim), requires_grad = True)
optimizer = optim.Adam([z], lr=0.09)
for t in range(10):
    print('-' * 50)
    optimizer.zero_grad()
    x_var = G(z).reshape(d_image)
    y_var = torch.matmul(A,x_var)
    loss = mse(y_var, y)
    print(t, loss)
    loss.backward(retain_graph=True)
    optimizer.step()
print('end epoch')

x_hat = G(z)
#x_hat.reshape(x.size(0),x.size(1))
grid = torchvision.utils.make_grid(x_hat.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())
# plt.show()
#print(Ameas.shape)
# measurements : y = A*x
#img_var_meas = torch.matmul(Ameas_var,img_var.to(device).reshape(d_image,1))



## Using Projected GD

In [ ]:
G = model.netG
mse = torch.nn.MSELoss()

G0 = G(torch.zeros(1,model.config.noiseVectorDim))
d_image = G0.numel()
x = G0.reshape(d_image)

f = 0.2 #compression rate
print('Compression rate is ', f)
m_image = int(f*d_image)
print('Number of measurements is ',m_image, ' for signal of length ', d_image)


# random Gaussian measurement matrix : A
A = torch.randn(m_image, d_image)
y = torch.matmul(A,x)

x_var = torch.zeros(d_image)
y_var = torch.zeros(m_image)

z = Variable(torch.randn(1,model.config.noiseVectorDim), requires_grad = True)
print('start')
# optimization min ||AG(z)-AG0||

for t in range(10):
    print('-' * 50)
    
    lr = 0.05
    v = x - lr*torch.matmul(A.transpose(), torch.matmul(A,x)-y)
    
    # minimize ||G(z)-v||
    optimizer = optim.Adam([z], lr=0.09)
    for i in range(10):
        optimizer.zero_grad()
        loss = mse(G(z), v)
        print(t, loss)
        loss.backward(retain_graph=True)
        optimizer.step()
        print('end epoch')

x_hat = G(z)
#x_hat.reshape(x.size(0),x.size(1))
grid = torchvision.utils.make_grid(x_hat.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())

## A=I. Recover G0 (naive)

In [ ]:
from torch.autograd import Variable
import torch.optim as optim

G = model.netG
mse = torch.nn.MSELoss()



G0 = G(torch.zeros(1,model.config.noiseVectorDim))

z = Variable(torch.randn(1,model.config.noiseVectorDim), requires_grad = True)
gen_image = G(z)
grid = torchvision.utils.make_grid(gen_image.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())
plt.show()


optimizer = optim.Adam([z], lr=0.09)
for t in range(10):
    print('-' * 50)
    optimizer.zero_grad()
    x = G(z)
    loss = mse(x,G0)
    print(t, loss)
    loss.backward(retain_graph=True)
    optimizer.step()
print('end epoch')
    
optimizer = optim.Adam([z], lr=0.06)
for t in range(10): 
    print('-' * 50)
    optimizer.zero_grad()
    x = G(z)
    loss = mse(x,G0)
    print(t, loss)
    loss.backward(retain_graph=True)
    optimizer.step()
print('end epoch')

optimizer = optim.Adam([z], lr=0.03)
for t in range(10): 
    print('-' * 50)
    optimizer.zero_grad()
    x = G(z)
    loss = mse(x,G0)
    print(t, loss)
    loss.backward(retain_graph=True)
    optimizer.step()
print('end epoch')

optimizer = optim.Adam([z], lr=0.01)
for t in range(10): 
    print('-' * 50)
    optimizer.zero_grad()
    x = G(z)
    loss = mse(x,G0)
    print(t, loss)
    loss.backward(retain_graph=True)
    optimizer.step()


In [ ]:
gen_image = G(z)


grid = torchvision.utils.make_grid(gen_image.clamp(min=-1, max=1), scale_each=True, normalize=True)
plt.imshow(grid.detach().permute(1, 2, 0).cpu().numpy())



In [ ]:
import time
import torch


t0 = time.time()



m = 5000
d = 20000
A = torch.randn(m,d)
x = torch.randn(d)
y = torch.matmul(A,x)
print(y.size())




t1 = time.time()
print('\ntime elapsed:',t1-t0)

In [1]:
import time
import torch

t0 = time.time()

x = torch.randn(50000)
A = torch.randn(10000,50000)
y = torch.matmul(A,x)

t1 = time.time()
print('\ntime elapsed:',t1-t0)


time elapsed: 77.59119868278503
